# Hidding the network jitter

Maximilian Hebeis, Artur Jakubowski, Weronika Niezorawska, Julia Plichta

We choose a simulated link option, so we are using two separete instances of intermcoms that runs in two terminals
 
 ## Measuring latency
 
 We are sending the ping to the localhost where chunksize is set to 1024, using the specified command and we are saving the output in a ping.txt file:
 
 **`ping localhost -c 100 -s <payload_length_in_bytes> > /tmp/ping.txt`**

 >PING localhost(localhost (::1)) 1024 data bytes  
 1032 bytes from localhost (::1): icmp_seq=1 ttl=64 time=0.054 ms  
 1032 bytes from localhost (::1): icmp_seq=2 ttl=64 time=0.052 ms  
 1032 bytes from localhost (::1): icmp_seq=3 ttl=64 time=0.065 ms  
 1032 bytes from localhost (::1): icmp_seq=4 ttl=64 time=0.079 ms<  
 ...

##### localhost ping statistics
100 packets transmitted, 100 received, 0% packet loss, time 100267ms  
rtt min/avg/max/mdev = 0.038/0.056/0.128/0.012 ms  
  
  
## Bulding histogram with python
Now we are calculating the average latency, the maximum absolute deviation (jitter) and the Pearson correlation coefficient using this Python code from the website:

In [4]:
import numpy as np 
from scipy import stats

latencies = np.loadtxt("/tmp/localhost_latencies.txt") 
average_latency = np.average(latencies) 
print("\naverage latency =", average_latency) 
max_latency = np.max(latencies) 
min_latency = np.min(latencies) 
maximum_absolute_deviation = max(max_latency - average_latency, average_latency - min_latency) 
print("maximum absolute deviation (jitter) =", maximum_absolute_deviation) 
correlation_coefficient = stats.pearsonr(latencies, np.roll(latencies, 1))[0] 
print("Pearson correlation coefficient =", correlation_coefficient) 
if correlation_coefficient < 0: 
  print("Correlation coefficient < 0: use 0 (no correlation between RTT samples) in your experiments") 
histogram = np.histogram(latencies) 
np.savetxt("/tmp/localhost_histogram.txt", histogram[0])


 


average latency = 0.02862
maximum absolute deviation (jitter) = 0.039880000000000006
Pearson correlation coefficient = 0.15588872533615594


**`plot "/tmp/localhost_histogram.txt" with histogram`**

  
  
## Characterizing the latency in the internet
We were using 'eduroam' network to conduct an experiment.    
- Our local IP of our router is: `172.20.32.1`  
- And public IP is: `150.214.223.70`

Unfortunetly we could not receive any ping answer from our router by sending it using it's public address.

> PING 150.214.223.70 (150.214.223.70) 1024(1092) bytes of data.  

>--- 150.214.223.70 ping statistics ---  
>100 packets transmitted, 0 received, 100% packet loss, time 101383ms

## Quantification of the QoE (In our host, simulating the Internet)

Using the data we obtained (average latency, max. absolute deviation (jitter) and the Pearson correlation coefficient), we simulate the measured latency using `tc` by adding a new rule for the loopback device `lo`.

Connecting to `localhost` with `minimal.py`, the network jitter is clearly audible:

<iframe width="560" height="315"
 src="https://youtube.com/embed/r1WWVCR-OfQ">
</iframe>

Following this, we tried buffering times of 10, 50, 70, 80, 85 and 90 ms and determined a buffering time of 85ms to be the minimum buffering time that allows us to hide the jitter in our network environment.

After running `buffer.py` with a `buffering_time` of 85 ms, the jitter is gone:

<iframe width="560" height="315"
 src="https://youtube.com/embed/EWQKjyk6SsA">
</iframe>